In [ ]:
!pip install -r DSAN-5800-LoRA-Project/requirements.txt

In [ ]:
# Clone your repo (or upload files)
!git clone https://github.com/mandyjsun/DSAN-5800-LoRA-Project.git
%cd DSAN-5800-LoRA-Project

In [ ]:
# Step 1: Prepare data
!mkdir -p data/processed artifacts/checkpoints artifacts/metrics
!python scripts/prepare_data.py \
  --out-dir data/processed \
  --train-ratio 0.8 --val-ratio 0.1 --test-ratio 0.1

In [ ]:
# Step 2: Baseline inference (zero-shot)
!python scripts/baseline_infer.py \
  --model mistralai/Mistral-7B-Instruct-v0.2 \
  --mbpp-file data/processed/mbpp_test.jsonl \
  --out-file artifacts/metrics/baseline_generations.jsonl

In [ ]:
# Step 3: Train LoRA (rank 8)
!python scripts/train_lora.py \
  --config configs/lora_r8.yaml \
  --train-file data/processed/train.jsonl \
  --val-file data/processed/val.jsonl \
  --output-dir artifacts/checkpoints/mistral7b-code-r8

In [ ]:
# Step 4: Evaluate LoRA r=8 on MBPP
!python scripts/eval_mbpp.py \
  --base-model mistralai/Mistral-7B-Instruct-v0.2 \
  --adapter-path artifacts/checkpoints/mistral7b-code-r8 \
  --mbpp-file data/processed/mbpp_test.jsonl \
  --out-dir artifacts/metrics

In [ ]:
# Step 5: Train LoRA (rank 32) - OPTIONAL (uses more VRAM)
!python scripts/train_lora.py \
  --config configs/lora_r32.yaml \
  --train-file data/processed/train.jsonl \
  --val-file data/processed/val.jsonl \
  --output-dir artifacts/checkpoints/mistral7b-code-r32

In [ ]:
# Step 6: Evaluate LoRA r=32 on MBPP
!python scripts/eval_mbpp.py \
  --base-model mistralai/Mistral-7B-Instruct-v0.2 \
  --adapter-path artifacts/checkpoints/mistral7b-code-r32 \
  --mbpp-file data/processed/mbpp_test.jsonl \
  --out-dir artifacts/metrics



In [ ]:
# Download results
from google.colab import files
!zip -r artifacts.zip artifacts/
files.download('artifacts.zip')